<a href="https://colab.research.google.com/github/daffamaul30/Naive-Bayes/blob/master/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import seaborn as sn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

## Mount & Read File From Drive

Bisa dirubah jika tidak ingin membaca file dari drive dengan :


dataset = pd.read_csv("path or filename(jika dalam satu folder)")


In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')
dataset = pd.read_csv("gdrive/My Drive/Colab Notebooks/fifa20.csv")

## Mengambil kolom/fitur yang diperlukan saja. Berdasarkan Data Exploration

In [0]:
dataset = dataset[['skill_dribbling','skill_curve','skill_ball_control','skill_moves']]

In [4]:
dataset.head()

,skill_dribbling,skill_curve,skill_ball_control,skill_moves
0,97,93,96,4
1,89,81,92,5
2,96,88,95,5
3,12,13,30,1
4,95,83,94,4


## Penentuan Label

menjadikan kolom "skill_moves" menjadi label

In [0]:
#skill_moves sbg label
label = dataset['skill_moves'].tolist()
dataset = dataset.drop(columns=['skill_moves'])
# print(dataset)

## Split Menjadi Train dan Test

dengan porsi Train 70% Test 30%

In [0]:
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=0.3, random_state = 0)

## Training Process

proses pembentukan model dengan metode Naive bayes

In [7]:
model = GaussianNB()
model.fit(x_train,y_train)

y_predict = model.predict(x_test)

# print(y_predict)
y_predict

array([3, 2, 2, ..., 3, 3, 1])

## EVALUASI

Confusion Matrix

In [21]:
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(y_predict, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)

df_confusion

Predicted,1,2,3,4,5
Actual,,,,,
1,593,8,0,0,0
2,14,2078,537,1,0
3,0,193,1571,174,1
4,0,0,123,167,13
5,0,0,0,7,4


In [10]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           1       0.98      0.99      0.98       601
           2       0.91      0.79      0.85      2630
           3       0.70      0.81      0.75      1939
           4       0.48      0.55      0.51       303
           5       0.22      0.36      0.28        11

    accuracy                           0.80      5484
   macro avg       0.66      0.70      0.67      5484
weighted avg       0.82      0.80      0.81      5484



In [8]:
print("Akurasi Model : ",metrics.accuracy_score(y_test,y_predict))

Akurasi Model :  0.8047045951859956
